# Fine Tunning ChatGPT to Make Precise Youtube Chapters Sections

Let's look at the documentation on the [OPENAI website](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset) to learn how to fine tune a chatgpt model to a specific use case using a custom dataset.

Starting with the basics, we need an example format, which should be as such:

```
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
{"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}
```

So, to fine tune a ChatGPT model we'll need to create a dataset organized in the format shown above. Essentially we'll have a .json file with several (as many as the number of inputs for the fine tunning) dictionaries that have this format of:

```
{"<messages key>"}: [{"role": "system", "<content key>": "<some text content>"},
{"<messages key>"}: {"role": "user", "<content key>": "<some text content>"},
{"<messages key>"}: {"role": "assistant", "<content key>": "<some text content>"}]
```

Here we'll assume that we will only fine tune actual ChatGPT, so we won't cover the formats for models like 'babbage-002' and others.

Ok, given this format, let's create our dataset.

Quote from the actual documentation:


> To fine-tune a model, you are required to provide at least 10 examples. We typically see clear improvements from fine-tuning on 50 to 100 training examples with gpt-3.5-turbo but the right number varies greatly based on the exact use case.



Let's try to get at least 30 examples for our custom dataset.

Just as in regular fine tunning with any Machine Learning model, its advisable to get stablish a train-test split so that you are on top of your model's performance.

Also, its extremely important to be aware of token limits and token costs, given that this will be a paid API.

Each training example will be limited to the context length of ChatGPT (therefore 4096 tokens), so its good to set some good practices in place to avoid issues with large inputs, the [OpenAI documentation](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset#:~:text=Each%20training%20example,the%20OpenAI%20cookbook.) recommends you check that the total token count in the message contents are under 4000 as a good rule of thumb.

Let's first create this dataset programatically using GPT4

In [32]:
import openai
import tiktoken
from IPython.display import display, Markdown

def get_response(prompt_question,):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "system", "content": "You are a helpful research and\
            programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response["choices"][0]["message"]["content"]

def get_num_tokens(prompt, model="gpt-3.5-turbo"):
    """Calculates the number of tokens in a text prompt"""    

    enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

    return len(enc.encode(prompt))

Great! Now, let manually inspect the flashcards generated with gpt4 to select the best ones for our custom training dataset.

In [35]:
prompt = f"Create 15 examples of 1 paragraph first person human-like descriptions of tasks and goals for the upcoming weeks. Each example should be in a bullet point."

task_paragraphs = get_response(prompt)

display(Markdown(task_paragraphs))

- I need to pick up the pace on my academic research assignment. I'll be spending the next few weeks rigorously scrutinizing the sources, journal articles, and additional materials I've collected. My aim is to understand the relationships in the data and eventually find a significant inference.

- I am planning on developing my own website in the coming weeks. It's going to be an informative platform about climate change. I will be using HTML, CSS, and JavaScript for this. My goal is to learn more in the process and create an engaging user interface.

- I’m working on getting my finances straightened out. This includes setting up a detailed budget, monitoring my spending habits, and adjusting where necessary. Plus, my goal for the next few weeks is to be more mindful about where my money goes and ideally, put some aside for saving.

- I’m planning to run a full marathon in the next 3 months, so the next weeks will be full of intense training sessions. My goal is to improve my stamina and ensure my body is prepared for the endurance race.

- I am aiming to learn a new language, French, over the course of the next few weeks. My strategy is to dedicate at least an hour each day for this self-paced learning. A new language skill will definitely enhance my personal and professional competencies.

- I'm working on a new art project. This involves creating a series of paintings aimed at expressing different human emotions. Over the next few weeks, my goal is to complete at least five of these artworks.

- Over the next few weeks, I will be preparing to present my research at a major conference. My focus will be on refining my presentation and making sure that all the information is accurate and up-to-date.

- As a chef, the upcoming weeks will see me experimenting with new recipes and preparing a menu for the opening of my restaurant. My aim is to curate an appealing balance between traditional and contemporary flavors.

- I am aiming to complete the first draft of my novel in the next couple of weeks. The main goal is to concentrate on character development and plot progression, ensuring that the storyline remains engaging for the readers.

- I am preparing for my thesis defense due in two weeks. Most of my time will be devoted to mastering my presentation, seeking the inputs from my guides, and getting ready to answer any tricky queries thrown at me.

- My plan for the next few weeks is to learn coding concepts and programming languages. I am starting with Python and then moving on to JavaScript.  

- Ahead of me in the upcoming weeks is a web design course. My focus will be not only on mastering the theoretical aspects but also on acquiring hands-on experience. My final goal is to create an aesthetic and fully functional website prototype.

- I am planning to read several books on personal development in the upcoming weeks. Each represents a different aspect that I aim to improve, such as leadership skills, emotional intelligence, and money management.

- For the next few weeks, I'm preparing for my upcoming backpacking trip. I'll be planning the route, gathering the necessary supplies, and researching potential shelters along the way. 

- My professional certification exam is due in a month, so the upcoming weeks will be dedicated to thorough revision and practice. I am aiming to ace it this time, enhancing my career prospects.

In [36]:
task_list_prompts = task_paragraphs.split("\n")

task_list_prompts

["- I need to pick up the pace on my academic research assignment. I'll be spending the next few weeks rigorously scrutinizing the sources, journal articles, and additional materials I've collected. My aim is to understand the relationships in the data and eventually find a significant inference.",
 '',
 "- I am planning on developing my own website in the coming weeks. It's going to be an informative platform about climate change. I will be using HTML, CSS, and JavaScript for this. My goal is to learn more in the process and create an engaging user interface.",
 '',
 '- I’m working on getting my finances straightened out. This includes setting up a detailed budget, monitoring my spending habits, and adjusting where necessary. Plus, my goal for the next few weeks is to be more mindful about where my money goes and ideally, put some aside for saving.',
 '',
 '- I’m planning to run a full marathon in the next 3 months, so the next weeks will be full of intense training sessions. My goal 

In [37]:
for t in task_list_prompts:
    print(t)

- I need to pick up the pace on my academic research assignment. I'll be spending the next few weeks rigorously scrutinizing the sources, journal articles, and additional materials I've collected. My aim is to understand the relationships in the data and eventually find a significant inference.

- I am planning on developing my own website in the coming weeks. It's going to be an informative platform about climate change. I will be using HTML, CSS, and JavaScript for this. My goal is to learn more in the process and create an engaging user interface.

- I’m working on getting my finances straightened out. This includes setting up a detailed budget, monitoring my spending habits, and adjusting where necessary. Plus, my goal for the next few weeks is to be more mindful about where my money goes and ideally, put some aside for saving.

- I’m planning to run a full marathon in the next 3 months, so the next weeks will be full of intense training sessions. My goal is to improve my stamina a

In [44]:
task_steps = []
for t in task_list_prompts:
    prompt = f"Given this paragraph description of tasks and goals: \n '''{t}''' \n create a bullet point list with all the necessary steps to accomplish all of them."
    task_steps.append(get_response(prompt))

Let's evaluate the dataset generated programatically:

In [45]:
for t in task_steps:
    display(Markdown(t))

1. **Planning and Scheduling:** Develop a detailed plan and schedule for the next few weeks. This should include targets for completing specific sections of the research.

2. **Organizing sources and materials:** For effective and quick retrieval, organize all the accumulated sources, journal articles, and additional materials coherently.

3. **Comprehensive Analysis:** Rigorously scrutinize every source, journal article, and additional material. 

4. **Data Interpretation and Visualization:** Use analytical methods to interpret data, observing patterns and trends from the raw data. Consider various data visualization tools to make the interpretation easier.

5. **Drawing Relationships:** Study the data to understand relationships between different variables. This could involve using statistical tools to determine correlations or causal relationships.

6. **Finding Significant Inference:** Based on the relationships drawn from the data, arrive at a significant conclusion or inference.

7. **Documentation:** Document all findings and the process in an organized and easy-to-understand manner. The documentation process may also involve writing an academic paper with the findings.
   
8. **Editing and Proofreading:** Ensure the final document or thesis is checked for any errors in spelling, grammar, syntax and format. 

9. **Preparation for Presentation:** Prepare a presentation of the findings, focusing on the significant inference and the process followed to reach it.
  
10. **Evaluation of Final Work:** Review the completed work to ensure that all goals and requirements were met. If necessary, make any final adjustments.

Apologies for the confusion, but you seem to have left the task description empty. Could you please provide a brief description of the tasks and goals you want to accomplish? This will allow me to create an accurate and helpful list for you.

1. Research on Website Development:
   - Understand the basics of HTML, CSS, and JavaScript if not already familiar.
   - Explore and Analyze existing informative platforms about climate change for inspiration.

2. Planning:
   - Define the purpose of the website.
   - Decide on the amount and type of content to be placed on the website.
   - Sketch a crude layout of the website's user interface based on the content.

3. Learning about Climate Change:
   - Conduct research on climate change and gather credible sources.
   - Plan on how to present the information in an engaging way.

4. Web Development:
   - Begin coding the basic structure of the webpages using HTML.
   - Design the layout of the site using CSS templates.
   - Add interactive elements with JavaScript.

5. Testing and Debugging:
   - Test the website on various browsers and devices to ensure its functionality.
   - Debug any issues or problem areas in the code.

6. User Interface Design:
   - Design the user interface that is engaging and easy to navigate.
   - Incorporate interactive features like sliders, pop-ups, or responsive icons to create a dynamic UI.
   - Use best web design principles to ensure usability and accessibility.

7. Content Creation:
   - Write or curate the informative content about climate change.
   - Make sure to structure the content in an easy-to-read and engaging format.
   - Include visual elements like infographics, images or videos where suitable.

8. Review and Iterations:
   - Review the website design, functionalities, and content. 
   - Make any necessary iterations to improve user interface or content.

9. Deployment:
   - Choose a suitable domain name and a web hosting service.
   - Deploy your website and make it live.

10. Promotion and Maintenance:
    - Promote the website on different platforms to reach out to your target audience.
    - Regularly check and update the website content, design, code, and other performances to ensure everything works properly. 

Remember, learning is a key goal here. So, make sure to document your challenges, victories, and lessons during this journey.

You haven't provided any specific paragraph description of tasks and goals. Please provide the detailed tasks and their objectives so I can assist in breaking them down into manageable, ordered steps.

1. Obtain a complete overview of current financial status:
   - Compile all bank statements, balances, outstanding debts, and income sources.
   - Detail any regular expenses, both necessary (rent/mortgage, utilities, food) and discretionary (entertainment, eating out).

2. Set up a detailed budget: 
   - List all income sources.
   - List all regular expenses.
   - Allocate portions of income for each regular expense.
   - Include a section in the budget for savings.

3. Monitor spending habits:
   - Take note of every single expense over a set period, e.g., one month.
   - Try to classify each expense into a budget category.
   - Review this tracking regularly to understand patterns and any unplanned spending.

4. Adjust spending and budget where necessary:
   - Identify areas of overspending and develop plans to curb that.
   - Reallocate budget funds as necessary based on tracking of expenditure.
   - Update the budget regularly based on changes in income and expenses.

5. Be more mindful about where money goes:
   - Continue tracking all expenses.
   - Regularly review and reflect on the necessity/value of each expenditure.
   - When considering a purchase, take a moment to consider the budget and whether the expense is planned/affordable.
   
6. Start a savings plan:
   - Determine a reasonable amount to save from each paycheck, taking into account the detailed budget and spending adjustments.
   - Set up an automatic transfer to a savings account on each payday to ensure the savings happen.
   - Monitor the growth of the savings fund regularly.
   - Keep the savings in a separate account or other inaccessible form to avoid the temptation to spend it.

7. Set financial goals:
   - Decide on short-term and long-term financial goals.
   - Make sure these goals are reflected in your budget – you may need to adjust spending in some categories to allow saving for these goals. 

8. Continue to review the budget and spending regularly – at least monthly – to keep track of your progress towards your financial goals. 

9. Look into financial advice or assistance if required – if finding the budgeting or saving aspect tough to handle, consider reaching out to a financial advisor or using a financial planning app.

...

1. Identify Specific Books: Begin by selecting the desired books for each particular skill set you wish to develop. Look for books that have high ratings or are recommended by experts in each field.

2. Set a Reading Schedule: Create a specific reading schedule. Dedicate a certain amount of time each day for reading. The schedule can include one book at a time or rotate between the books, depending on your preference.

3. Purchase or Borrow Books: Once you have a list, purchase or borrow the books from a library or online platform.

4. Note Taking: As you read, keep a note pad or use an online note taking service to jot down important points, insights, and any practice exercises presented in the books.

5. Reflect and Practice: Take the time to reflect on what you have learned from each book. Practice the concepts discussed as much as possible to allow the knowledge to integrate into your daily living.

6. Join Online Discussions/Book Clubs: Join online discussions or book clubs focusing on personal development. This will allow the concepts you learned to be explored further, and you can gain different perspectives.

7. Apply to Real-Life Situations: Start using the strategies and lessons in real-life situations to improve your leadership skills, emotional intelligence, and money management.

8. Evaluate Progress: After practicing the methods learned, evaluate your progress. Note any improvements or obstacles to address.

9. Seek Additional Tools/Resources: After having read and started practicing, you may identify areas where you need further assistance. Look for additional resources such as articles, courses, online tools, or coaching to support these areas.

10. Repeat the Process: Once you've completed and mastered one area, move on to the next book. Also, continually revisit previous books to refresh your knowledge and integrate the learned skills into your daily life.

Sorry for the confusion, but I can't generate a list of necessary steps because the description of tasks and goals you mentioned isn't included in your request. Could you please provide me with the details? I'd be glad to help you with your request once I have more information.

1. Select Desired Destination: 
   - Research potential destinations suitable for backpacking.
   - Evaluate the destinations based on factors such as distance, landscape, weather conditions, and personal preference.

2. Plan the Route:
   - Investigate the specific trails and terrains available at the chosen destination.
   - Decide on a route considering the difficulty level and physical fitness.
   - Ensure to mark resting points, potential shelters, and emergency exits.

3. Gather Necessary Supplies:
   - Research and prepare a list of items required for backpacking.
   - Check existing gear and determine what additional equipment is needed.
   - Prioritize the list based on necessity: backpack, tent, sleeping bag, clothing, food and water supplies, cooking tool, navigation aids etc.
   - Purchase or borrow missing items.

4. Research Potential Shelters:
   - Look up forests, national parks, campsites, and cabins that allow overnight stay.
   - Consider safety, weather resistance, availability and proximity to the backpacking route.
   - Contact owners or administrators of potential shelters for availability and rules.

5. Physical Preparation:
   - Begin a fitness routine suitable for the trip if you don't have one already.
   - Consult with a doctor for health clearance if necessary.

6. Inform Someone About Your Plan:
   - Share specific details of the planned route with a trusted contact.
   - Discuss estimated times of departure and arrival, and the potential shelters.

7. Test Gear Before the Trip:
   - Test all equipment to ensure it is working condition.
   - Learn how to use new equipment or gear.

8. Prepare First-aid and Emergency Supplies:
   - Gather and pack first-aid essentials.
   - Learn essential first-aid skills if you don't already have them.
   - Pack any necessary emergency supplies such as a flashlight, whistle, and multi-tool.

9. Pack Food and Water Supplies:
   - Plan meals for each day of the trip.
   - Carry portable water filters or purification tablets for safe drinking water.

10. Final Check:
    - Review all preparations one last time.
    - Check weather updates and any changes in the destination area.

I apologise for the misunderstanding, but I'll need the actual paragraph description of tasks and goals to breakdown. Could you please provide the necessary information? I'll be more than happy to help create the bullet point list afterwards.

1. Determine the syllabus and topics covered in the professional certification examination.
2. Gather all the necessary study material and resources for each topic.
3. Develop a study schedule that accounts for the time remaining before the examination; make sure to include breaks for rest and refreshment.
4. Allocate more revision time to areas you find challenging or where you scored low in past attempts.
5. Regularly practice with past or sample examination papers to gain familiarity with question formats and time management.
6. Set weekly goals and track your progress to ensure you are meeting your study targets.
7. Look for online forums or study groups where you can discuss and resolve your doubts and learn through the experience of others.
8. Take care of your health by maintaining a balanced diet, exercising regularly, and getting sufficient sleep; this is important for optimum brain function and concentration.
9. Prepare a checklist of all items you need to take to the exam, such as ID, stationery, etc.
10. Review the examination rules and regulations to avoid any penalties or disqualifications.
11. A few days before the exam, start tapering down your study to avoid burnout.
12. A day before the exam, relax and get a good night's sleep to be fresh and alert on the exam day.
13. On the day of the exam, have a light meal, reach the exam venue in advance and stay calm and focused during the exam.
14. After the exam, review your performance and note down areas for improvement for future reference. Don't forget to update your CV and LinkedIn profile once you've died your certification.

In [48]:
for desc,steps in zip(task_list_prompts, task_steps):
    print(desc) 
    print(steps)
    print("************")   

- I need to pick up the pace on my academic research assignment. I'll be spending the next few weeks rigorously scrutinizing the sources, journal articles, and additional materials I've collected. My aim is to understand the relationships in the data and eventually find a significant inference.
1. **Planning and Scheduling:** Develop a detailed plan and schedule for the next few weeks. This should include targets for completing specific sections of the research.

2. **Organizing sources and materials:** For effective and quick retrieval, organize all the accumulated sources, journal articles, and additional materials coherently.

3. **Comprehensive Analysis:** Rigorously scrutinize every source, journal article, and additional material. 

4. **Data Interpretation and Visualization:** Use analytical methods to interpret data, observing patterns and trends from the raw data. Consider various data visualization tools to make the interpretation easier.

5. **Drawing Relationships:** Study 

In [49]:
empty_indices = [i for i, x in enumerate(task_list_prompts) if not x]
print(empty_indices)

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27]


In [55]:
# Define the indexes to remove
indexes_to_remove = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27]

# Remove the elements with the given indexes from task_list_prompts and task_steps
task_list_prompts = [t for i, t in enumerate(task_list_prompts) if i not in indexes_to_remove]
task_steps = [t for i, t in enumerate(task_steps) if i not in indexes_to_remove]

In [ ]:
# prompt_to_gpt4 = """Consider this format for a dataset for the chatgpt fine tuning api:
# '''
# {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."}]}
# {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "Oh, just some guy named William Shakespeare. Ever heard of him?"}]}
# {"messages": [{"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "Around 384,400 kilometers. Give or take a few, like that really matters."}]}
# '''

# Write the python code to create a dataset like this one programatically given that the system prompt will always be: "You are a helpful planning assistant"
# and that the user prompts will be inside a list called 'task_list_prompts' and the assistant content will be inside a list called 'task_steps'."""

In [58]:
import json

# Ensure the lists have the same length
assert len(task_list_prompts) == len(task_steps), "Mismatched lengths between prompts and responses"

# Creating the dataset
dataset = []
system_prompt = "You are a helpful planning assistant"

for user_content, assistant_content in zip(task_list_prompts, task_steps):
    interaction = {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": assistant_content}
        ]
    }
    dataset.append(interaction)

In [59]:
dataset

[{'messages': [{'role': 'system',
    'content': 'You are a helpful planning assistant'},
   {'role': 'user',
    'content': "- I need to pick up the pace on my academic research assignment. I'll be spending the next few weeks rigorously scrutinizing the sources, journal articles, and additional materials I've collected. My aim is to understand the relationships in the data and eventually find a significant inference."},
   {'role': 'assistant',
    'content': '1. **Planning and Scheduling:** Develop a detailed plan and schedule for the next few weeks. This should include targets for completing specific sections of the research.\n\n2. **Organizing sources and materials:** For effective and quick retrieval, organize all the accumulated sources, journal articles, and additional materials coherently.\n\n3. **Comprehensive Analysis:** Rigorously scrutinize every source, journal article, and additional material. \n\n4. **Data Interpretation and Visualization:** Use analytical methods to int

In [60]:
# Writing to a file
with open('dataset.jsonl', 'w') as f:
    for entry in dataset:
        f.write(json.dumps(entry) + '\n')

# Now 'dataset.jsonl' will contain the dataset in the desired format.

In [1]:
# source: https://cookbook.openai.com/examples/chat_finetuning_data_prep
import json
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [61]:
data_path = "./dataset.jsonl"

# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Initial dataset stats
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 15
First example:
{'role': 'system', 'content': 'You are a helpful planning assistant'}
{'role': 'user', 'content': "- I need to pick up the pace on my academic research assignment. I'll be spending the next few weeks rigorously scrutinizing the sources, journal articles, and additional materials I've collected. My aim is to understand the relationships in the data and eventually find a significant inference."}
{'role': 'assistant', 'content': '1. **Planning and Scheduling:** Develop a detailed plan and schedule for the next few weeks. This should include targets for completing specific sections of the research.\n\n2. **Organizing sources and materials:** For effective and quick retrieval, organize all the accumulated sources, journal articles, and additional materials coherently.\n\n3. **Comprehensive Analysis:** Rigorously scrutinize every source, journal article, and additional material. \n\n4. **Data Interpretation and Visualization:** Use analytical methods to interp

Perfect! Now lets estimate some token counts and costs.

We'll be checkinf for this validation checklist as described in the [tutorial suggested by the OpenAI docs](https://cookbook.openai.com/examples/chat_finetuning_data_prep#:~:text=Format%20validation,for%20easier%20debugging.).  

In [64]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [65]:
# Some helpful utilities

encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

Data warnings and token counts.

In [66]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 304, 531
mean / median: 427.53333333333336, 424.0
p5 / p95: 362.2, 503.6

#### Distribution of num_assistant_tokens_per_example:
min / max: 241, 452
mean / median: 361.8666666666667, 362.0
p5 / p95: 297.6, 434.2

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


As we can see from the output obtained, all of our examples are under the context length, which means they won't have to be truncated which is great news! :)

Cost estimation

Let's estimate the cost of this fine tunning based on the number of tokens in our dataset.

In [67]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~6413 tokens that will be charged for during training
By default, you'll train for 6 epochs on this dataset
By default, you'll be charged for ~38478 tokens


Ok, as we can see it seems like this fine tunning is not going to cost that much, given that if I check on the [pricing page](https://openai.com/pricing) for the cost associated with this amount of tokens:

![](2023-10-16-17-22-32.png)


Let's write a little function to calculate these costs automatically:

In [75]:
def calculate_cost_for_fine_tunning(token_count):
    return (0.008*token_count)/1000

In [76]:
calculate_cost_for_fine_tunning(38478)

0.307824

Ok, so it seems that the amount will be 0.30 cents which seems reasonable, although we should take into account the fact that we'll be charged more for using this fine tuned model as well, and we only used 10 examples for this use case!

Now lets upload our newly generated dataset file!

To do that part, lets write a simply Python script to upload the file, we'll use this snippet from the [OpenAI API docs](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset#:~:text=import%20os%20import%20openai%20openai.api_key%20%3D%20os.getenv(%22openai_api_key%22)%20openai.file.create(%20file%3Dopen(%22mydata.jsonl%22%2C%20%22rb%22)%2C%20purpose%3D'fine-tune'%20)).

In [2]:
import os
import openai

openai.File.create(
  file=open("./dataset.jsonl", "rb"),
  purpose='fine-tune'
)

<File file id=file-ICGS6dyaltzMJuXdnHcpjGTr at 0x12d297590> JSON: {
  "object": "file",
  "id": "file-ICGS6dyaltzMJuXdnHcpjGTr",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 34150,
  "created_at": 1697756044,
  "status": "uploaded",
  "status_details": null
}

In [1]:
list_files = openai.File.list()

In [2]:
list_files

<OpenAIObject list at 0x107694d70> JSON: {
  "object": "list",
  "data": [
    {
      "object": "file",
      "id": "file-d3Qb6gOJDiI8BJnfYsmmIUyv",
      "purpose": "fine-tune",
      "filename": "./prompt_data.json",
      "bytes": 6807,
      "created_at": 1641411317,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-z7wvIoSwHXoWlEgtQX36uzz4",
      "purpose": "fine-tune-results",
      "filename": "compiled_results.csv",
      "bytes": 8688,
      "created_at": 1641424829,
      "status": "processed",
      "status_details": null
    },
    {
      "object": "file",
      "id": "file-MvhptLxVWAbPYSN3ZB8hAyN3",
      "purpose": "fine-tune",
      "filename": "./prompt_data.json",
      "bytes": 73697,
      "created_at": 1641424357,
      "status": "error",
      "status_details": "Invalid file format. Example 38 cannot be parsed. Error: line contains invalid json: Invalid control character at: line 1 column 770 (char 7

Now, finally, we create our fine-tuned model by running this:

In [3]:
openai.FineTuningJob.create(training_file="file-ICGS6dyaltzMJuXdnHcpjGTr", model="gpt-3.5-turbo")

<FineTuningJob fine_tuning.job id=ftjob-rPPzK29a1UkXFSHTdEQX4heY at 0x105eef050> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-rPPzK29a1UkXFSHTdEQX4heY",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1697756148,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-gpLJbCQWtORw077QTyeX1IVP",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-ICGS6dyaltzMJuXdnHcpjGTr",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}

Here the `training_file` parameter correspond to the file id generated when running the previous snippet. 


Now we wait for an email confirmation that will let us know when the training is done. Which given the size of this job, should be pretty quckly

Neat thing is that you can promatically query the status of your jobs:

In [14]:
# List 10 fine-tuning jobs
openai.FineTuningJob.list(limit=10)

# Retrieve the state of a fine-tune
openai.FineTuningJob.retrieve("ftjob-rPPzK29a1UkXFSHTdEQX4heY")

# Cancel a job
#openai.FineTuningJob.cancel("file-kXlvspUQRKFUWcrW5pgzy0PW")

# List up to 10 events from a fine-tuning job
#openai.FineTuningJob.list_events(id="file-kXlvspUQRKFUWcrW5pgzy0PW", limit=10)

<FineTuningJob fine_tuning.job id=ftjob-rPPzK29a1UkXFSHTdEQX4heY at 0x107696510> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-rPPzK29a1UkXFSHTdEQX4heY",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1697756148,
  "finished_at": 1697756446,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::8BWIRV1U",
  "organization_id": "org-gpLJbCQWtORw077QTyeX1IVP",
  "result_files": [
    "file-DpX9PVkO1Y9bIndbRBjADyUS"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-ICGS6dyaltzMJuXdnHcpjGTr",
  "hyperparameters": {
    "n_epochs": 6
  },
  "trained_tokens": 38298,
  "error": null
}

Now we can go to the platform to check out our fine tuned model or run inference from it using the API! Let's look at how to run our fine tuned model

IN the platform we see:

![](2023-10-20-00-02-25.png)

So, we can choose that model and see how it performs for a different new video.

Querying the model:
![](2023-10-20-00-05-50.png)

The response:

![](2023-10-16-17-49-24.png)

It looks pretty good! :) Let's compare with the output of regular ChatGPT

![](2023-10-16-17-50-32.png)

The fine tuned model is definitely more thourough than the regular ChatGPT model so I would call this a nice success! :)

Now, let's run inference with our fine tuned model!

In [16]:
def get_response(prompt_question,fine_tuned_model_id="ft:gpt-3.5-turbo-0613:personal::8BWIRV1U"):
    response = openai.ChatCompletion.create(
        model=fine_tuned_model_id,
        messages=[{"role": "system", "content": "You are a helpful research and\
            programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response["choices"][0]["message"]["content"]

get_response("Tomorrow I have to practice for my presentation at the live-training for O'Reilly at least a couple more times, then run through all the slides and notebooks to check everything is in order. After that I have Jiu Jitsu training and one more rehearsal before the live-training at 18:00")

'Based on the given schedule, I can propose the following plan:\n\n1. Morning:\n- Check and revise the presentation content.\n- Practice for the presentation at least twice.\n\n2. Afternoon:\n- Run through all the slides and notebooks to ensure everything is in order.\n- Take a break for Jiu Jitsu training.\n- After training, do one more practice session to refine the presentation.\n\n3. Evening:\n- Live-training at 18:00.\n\nLet me know if you need any further assistance or if you have specific requirements for the rehearsal or the presentation.'

In [17]:
openai.Model.delete("ft:gpt-3.5-turbo-0613:personal::8BWIRV1U")

<Model model id=ft:gpt-3.5-turbo-0613:personal::8BWIRV1U at 0x1076965d0> JSON: {
  "id": "ft:gpt-3.5-turbo-0613:personal::8BWIRV1U",
  "object": "model",
  "deleted": true
}

Success!!!!!